In [2]:
from pyltp import Segmentor # pyltp 提供分词、词性标注、命名实体识别、依存句法分析、语义角色标注等功能。
from pyltp import SentenceSplitter
from pyltp import Postagger
from pyltp import NamedEntityRecognizer
from pyltp import Parser
#from get_synonyms import load_synonyms
import os

In [3]:
def split_sentences(string): # 分句
    sen = SentenceSplitter.split(string)
    sentences = [s for s in sen if len(s) != 0]
    return sentences


In [5]:
def split_words(string): # 分词
    LTP_DATA_DIR = 'D:/NLP/ltp_data_v3.4.0' # Ltp模型目录的路径
    cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model') # 分词模型路径，分词模型名称是‘cws.model’
    
    segmentor = Segmentor()
    segmentor.load(cws_model_path)
    words = segmentor.segment(string)
    segmentor.release()
    
    #print('\t'.join(words))
    
    return list(words)

In [6]:
def get_sentence_pos(words): # words分词后的
    LTP_DATA_DIR = 'D:/NLP/ltp_data_v3.4.0' # Ltp模型目录的路径
    pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model') # 词性标注模型路径，分词模型名称是‘pos.model’
    
    postagger = Postagger()
    postagger.load(pos_model_path)
    
    postags = postagger.postag(words)
    postagger.release()
    
    #print('\t'.join(postags))
    
    return list(postags)

In [7]:
def NER(words, postags): # 命名实体识别
    LTP_DATA_DIR = 'D:/NLP/ltp_data_v3.4.0'
    ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')
    
    recognizer = NamedEntityRecognizer()
    recognizer.load(ner_model_path)
    
    netags = recognizer.recognize(words, postags)
    recognizer.release()
    
    #print('\t'.join(netags))
    
    return list(netags)

In [8]:

def dependency_parsing(words, postags): # 依存句法分析
    LTP_DATA_DIR = 'D:/NLP/ltp_data_v3.4.0'
    par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')
    
    parser = Parser()
    parser.load(par_model_path)
    
    arcs = parser.parse(words, postags) # 句法分析
    parser.release()
    
    #print("\t".join("%d: %s" % (arc.head, arc.relation) for arc in arcs))
    
    return list(arcs)

In [26]:
def who_say_what(sentence): # 对单句提取新闻人物和言论
    
    content = list()
    say = ['说','告诉','讲','表示','回应','呼吁','指出']
    #say = load_synonyms('synonyms.txt')
    
    words = split_words(sentence)
    postags = get_sentence_pos(words)
    netags = NER(words, postags)
    
    if any(x in netags for x in ['S-Nh', 'S-Ni', 'S-Ns']): # If the sentence contains someone's saying
        arcs = dependency_parsing(words, postags)
        arcs = [(i, arc) for i, arc in enumerate(arcs) if arc.relation == 'SBV']

        #print("\t".join("%d: %s" % (arc[1].head, arc[1].relation) for arc in arcs))

        for arc in arcs:
            verb_index = arc[1].head  # Get the index of 说
            subject_index = arc[0]  # Get the index of the subject who said
            #print(subject_index)
            if words[verb_index - 1] not in say:
                continue
            content.append((words[subject_index], words[verb_index - 1], ''.join(words[verb_index:])))

    return content

In [27]:
def get_speech(data): # Get someone's speech from a data
    result = list()
    for i in split_sentences(data):
        speech = who_say_what(i)
        if speech != []:
            result.extend(speech)
    
    return result

In [28]:
def get_speech_from_file(file_name): # Get someone's speech from a file
    result = list()
    with open(file_name,'r+', encoding = 'utf-8') as file:
        for line in file:
            speech = who_say_what(line)
            if speech != []:
                result.extend(speech)
    file.close()
    return result

In [29]:
data = """
    “六一”国际儿童节前夕，习近平总书记向全国各族少年儿童致以节日祝贺，勉励他们刻苦学习知识，坚定理想信念，磨练坚强意志，锻炼强健体魄，为实现中华民族伟大复兴的中国梦时刻准备着。总书记的深情寄语凝聚着对少年儿童的真挚关怀和殷切期待，温暖激励广大少年儿童树立远大目标、努力学习知识本领，将来担当起民族复兴大任。

六月一日是北京市高一、高二、初一、初二年级和小学六年级学生返校上课的第一天。回到久别的校园，学生们倍加珍惜时光，收到习爷爷的节日问候和祝福，让孩子们更是深受鼓舞。

总书记指出，在这次全国人民万众一心抗击新冠肺炎疫情的斗争中，广大少年儿童听从党和政府号召以实际行动支持抗疫斗争，展现了我国少年儿童的良好精神风貌。在前期战“疫”艰难时刻，14岁的武汉女孩孙婉清给在一线奋战的父亲写下“没有一个冬天不可逾越”的家书，感动了亿万网友。

习近平总书记指出，当代中国少年儿童既是实现第一个百年奋斗目标的经历者、见证者，更是实现第二个百年奋斗目标、建设社会主义现代化强国的生力军。培养好少年儿童是一项战略任务，事关长远。

在新疆第一所民办国家通用语言小学——乌什县依麻木镇国语小学，孩子们正在收看“放飞梦想、快乐成长”主题诗歌朗诵，孩子们纷纷表达要倍加珍惜学习时光，为未来攀登科学高峰打好基础。

在云南师范大学附属润城学校举行了“从小学先锋，长大做先锋”主题班会活动，积极引导学生爱国与奉献、担当与奋斗。

在对少年儿童的寄语中，习近平总书记专门对共青团、少先队组织如何更好为党的少年儿童事业作出新的更大的贡献提出了明确要求。各级团组织认真学习领会总书记寄语精神，决心以深化改革的务实举措培养能够担当民族复兴大任的时代新人。
    """
    

In [30]:
speech = get_speech(data)
print(speech)

[('总书记', '指出', '，当代中国少年儿童既是实现第一个百年奋斗目标的经历者、见证者，更是实现第二个百年奋斗目标、建设社会主义现代化强国的生力军。')]


In [ ]:
#待优化